In [1]:
import pandas as pd

In [68]:
df = pd.read_csv('RFM_ht_data.csv',  dtype = {'InvoiceNo': 'str', 'CustomerCode': 'str'}, parse_dates = ['InvoiceDate'])

In [128]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [70]:
df.describe(include = 'all', datetime_is_numeric=True)

,InvoiceNo,CustomerCode,InvoiceDate,Amount
count,332730,332730,332730,332730.000000
unique,332730,123733,NaN,NaN
top,D0711810200626,19057820,NaN,NaN
freq,1,204,NaN,NaN
mean,NaN,NaN,2020-09-15 16:35:03.846361856,1239.029737
min,NaN,NaN,2020-09-01 00:00:00,-8925.000000
25%,NaN,NaN,2020-09-08 00:00:00,350.000000
50%,NaN,NaN,2020-09-16 00:00:00,720.445000
75%,NaN,NaN,2020-09-24 00:00:00,1491.507500
max,NaN,NaN,2020-09-30 00:00:00,131874.000000


In [71]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [72]:
# находим последнюю дату в датасете
last_date = df.InvoiceDate.max()

In [129]:
# для каждого пользователя считаем дату последней покупки, количество покупок, сумму покупок
data = df.groupby('CustomerCode').agg(last_purchase = ('InvoiceDate', lambda x: last_date-x.max()),\
                                                        n_orders = ('InvoiceNo', 'nunique'),\
                                                        sum_purchases = ('Amount', 'sum'))

In [130]:
# выражаем таймдельту в днях
data.last_purchase = data.last_purchase.dt.days

In [131]:
# для каждого показателя находим процентили
procentiles = data.quantile([0.25, 0.5, 0.75], axis = 0)
procentiles

,last_purchase,n_orders,sum_purchases
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [136]:
# функция для оценки recency по таблице с процентилями
def Rclass(x, parameter_name, procentiles_table):
    if x <= procentiles_table.loc[0.25, parameter_name]:
        return 1
    elif x <= procentiles_table.loc[0.5, parameter_name]:
        return 2
    elif x <= procentiles_table.loc[0.75, parameter_name]:
        return 3
    else:
        return 4
    

In [137]:
data['recency'] = data.last_purchase.apply(Rclass, args = ('last_purchase', procentiles ))

In [138]:
# функция для оценки frequency и money по таблице с процентилями
def FMclass(x, parameter_name, procentiles_table):
    if x <= procentiles_table.loc[0.25, parameter_name]:
        return 4
    elif x <= procentiles_table.loc[0.5, parameter_name]:
        return 3
    elif x <= procentiles_table.loc[0.75, parameter_name]:
        return 2
    else:
        return 1

In [139]:
data['frequency'] = data.n_orders.apply(FMclass, args = ('n_orders', procentiles ))
data['money'] = data.sum_purchases.apply(FMclass, args = ('sum_purchases', procentiles ))

In [141]:
data['RFM_class'] = data.recency.astype('str') + data.frequency.astype('str') + data.money.astype('str')

In [142]:
data

,last_purchase,n_orders,sum_purchases,recency,frequency,money,RFM_class
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443
...,...,...,...,...,...,...,...
99099927,10,1,961.10,3,4,3,343
99099936,0,1,1521.78,1,4,3,143
99099959,8,2,1444.56,2,3,3,233
